In [ ]:
import itertools
from heapq import heappush, heappop

In [ ]:
class Graph:
    def __init__(self, adjacency_list):
        self.adjacency_list = adjacency_list

class Vertex:
    def __init__(self, value):
        self.value = value


class Edge:
    def __init__(self, distance, vertex):
        self.distance = distance
        self.vertex = vertex


def dijkstra(graph, start, end):
    previous = {v: None for v in graph.adjacency_list.keys()}
    visited = {v: False for v in graph.adjacency_list.keys()}
    distances = {v: float("inf") for v in graph.adjacency_list.keys()}
    distances[start] = 0
    queue = PriorityQueue()
    queue.add_task(0, start)
    path = []
    while queue:
        removed_distance, removed = queue.pop_task()
        visited[removed] = True
        if removed is end:
            while previous[removed]:
                path.append(removed.value)
                removed = previous[removed]
            path.append(start.value)
            print(f"shortest distance to {end.value}: ", distances[end])
            print(f"path to {end.value}: ", path[::-1])
            return

        for edge in graph.adjacency_list[removed]:
            if visited[edge.vertex]:
                continue
            new_distance = removed_distance + edge.distance
            if new_distance < distances[edge.vertex]:
                distances[edge.vertex] = new_distance
                previous[edge.vertex] = removed
                queue.add_task(new_distance, edge.vertex)
    return

class PriorityQueue:
    def __init__(self):
        self.pq = []
        self.entry_finder = {}
        self.counter = itertools.count()

    def __len__(self):
        return len(self.pq)

    def add_task(self, priority, task):
        if task in self.entry_finder:
            self.update_priority(priority, task)
            return self
        count = next(self.counter)
        entry = [priority, count, task]
        self.entry_finder[task] = entry
        heappush(self.pq, entry)

    def update_priority(self, priority, task):
        entry = self.entry_finder[task]
        count = next(self.counter)
        entry[0], entry[1] = priority, count

    def pop_task(self):
        while self.pq:
            priority, count, task = heappop(self.pq)
            del self.entry_finder[task]
            return priority, task


In [5]:
vertices = [ Vertex('A'), Vertex('B'), Vertex('C'), Vertex('D'), Vertex('E')]

A, B, C, D, E = vertices

adjacency_list = {
  A: [Edge(4, B), Edge(2, C)],
  B: [Edge(4, A), Edge(3, C), Edge(2, D), Edge(3, E)],
  C: [Edge(2, A), Edge(3, B), Edge(4, D), Edge(5, E)],
  D: [Edge(2, B), Edge(4, C), Edge(1, E)],
  E: [Edge(3, B), Edge(5, C), Edge(1, D)],
}

my_graph = Graph(adjacency_list)


dijkstra(my_graph, start=A, end=D)

shortest distance to D:  6
path to D:  ['A', 'C', 'D']
